In [1]:
from __future__ import division
import os
import re
import json
import shapefile
import numpy as np
import pandas as pd
from pyproj import Proj
from shapely.geometry import Point, Polygon
import folium

In [ ]:
def fronteira(setor):
    start, end = 0, len(setor.shape.points)
    if len(setor.shape.parts) > 1:
        end = setor.shape.parts[1]
    return setor.shape.points[start:end]

In [ ]:
def shape2json(sf, setores, outfile):
    myProj = Proj("+proj=utm +zone=23S, +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
    fields = sf.fields[1:]
    field_names = [field[0] for field in fields]

    data = []
    for el in setores:
        #atr = dict(zip(field_names, el.record))
        atr = {"CD_GEOCODI": el.record[field_names.index('CD_GEOCODI')]}
        geom = {'coordinates': [], 'type': 'Polygon'}
        front = fronteira(el)
        s = []
        for coord in front:
            p = myProj(*coord, inverse=True)
            s.append([p[0], p[1]])

        geom['coordinates'].append(s)
        data.append({"type": "Feature", "geometry": geom, "properties": atr})

    #for b in data:
    #    for key in field_names:
    #        if type(b['properties'][key]) is bytes:
    #            b['properties'][key] = b['properties'][key].decode("ISO-8859-1")
            
    with open(outfile, "w") as geojson:
        geojson.write(json.dumps({"type": "FeatureCollection", "features": data}, indent=2))

In [3]:
df = pd.read_csv("gente_quebrada_por_bike.csv", sep=';')

In [6]:
locations = df[['latitude', 'longitude']]
locationlist = locations.values.tolist()

In [ ]:
arq_name = "sp_setores_censitarios/35SEE250GC_SIR"

In [ ]:
shp = open(arq_name + ".shp", "rb")
dbf = open(arq_name + ".dbf", "rb")
shx = open(arq_name + ".shx", "rb")

In [ ]:
sf = shapefile.Reader(shp=shp, dbf=dbf, shx=shx)

In [ ]:
field_names = [field[0] for field in sf.fields[1:]]
setores = []
for setor in sf.shapeRecords():
    meso = setor.record[field_names.index("NM_MUNICIP")]
    meso = meso if type(meso) is not bytes else meso.decode("ISO-8859-1")
    if meso == "SÃO PAULO":
        setores.append(setor)

In [ ]:
shape2json(sf, setores, "out.json")

In [7]:
mapa = folium.Map(width=800, height=800, location=[-22, -51])

In [ ]:
folium.GeoJson(json.load(open("out.json"))).add_to(mapa)

In [10]:
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df['end'][point]).add_to(mapa)

In [11]:
mapa